<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/stamp_isoscape/raster_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob
import tensorflow as tf
import joblib

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

Cloning into 'ddf_common_stub'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.50 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (4/4), done.


interactive(children=(Text(value='', description='Email', placeholder='Enter email'), Text(value='', descripti…

In [3]:
import model
import raster
import pytest
import importlib

importlib.reload(model)
importlib.reload(raster)

def fake_brazil_bounds():
  return raster.Bounds(minx=-73.97513931345594,
                       maxx=-34.808472803053895,
                       miny=-33.73347244751509,
                       maxy=5.266527396029211,
                       pixel_size_x=0.04166666650042771,
                       pixel_size_y=-0.041666666499513144,
                       raster_size_x=937, raster_size_y=941)

def test_coords_to_indices():
  bounds = raster.Bounds(50, 100, 50, 100, 1, 1, 50, 50)
  x, y = raster.coords_to_indices(bounds, 55, 55)
  assert x == 45
  assert y == 5

  bounds = raster.Bounds(-100, -50, -100, -50, 1, 1, 50, 50)
  x, y = raster.coords_to_indices(bounds, -55, -55)
  assert x == 5
  assert y == 45

  bounds = raster.Bounds(-10, 50, -10, 50, 1, 1, 60, 60)
  x, y = raster.coords_to_indices(bounds, -1, 13)
  assert x == 37
  assert y == 9

  x, y = raster.coords_to_indices(fake_brazil_bounds(), -67.14342073173958, -7.273271869467912e-05)
  assert x == 131 # was: 132
  assert y == 163

def test_mount_gdrive_local():
  raster.GDRIVE_BASE = "/content/gdrive"
  raster.mount_gdrive()
  assert os.path.exists(raster.GDRIVE_BASE)

def test_is_valid_point():
  assert(not raster.is_valid_point(0, 0,
                                   raster.atmosphere_isoscape_geotiff()))
  assert(raster.is_valid_point(-2.5, -59,
                                   raster.atmosphere_isoscape_geotiff()))

def smoke_test_gen_isoscape():
  model_path = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/unit_tests/test_model.tf"
  transformer_path = "/content/gdrive/MyDrive/amazon_rainforest_files/variational/model/unit_tests/test_model.pkl"
  vi_model = model.TFModel(model_path, transformer_path)

  required_geotiffs = ["Mean Annual Precipitation"]
  run_id = 'raster_unit_test'

  raster.generate_isoscapes_from_variational_model(run_id, vi_model, required_geotiffs, 235, 234)

  generated_means_raster = raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=0)
  assert 25.29 == pytest.approx(generated_means_raster.value_at(-59, -2.5), 0.01)
  assert(not raster.is_valid_point(0, 0, generated_means_raster))

  generated_vars_raster = raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=1)
  assert 1.54 == pytest.approx(generated_vars_raster.value_at(-59, -2.5), 0.01)
  assert(not raster.is_valid_point(0, 0, generated_vars_raster))

  anim = raster.animate(raster.load_raster(raster.get_raster_path(run_id+".tiff"), use_only_band_index=0), 1, 1)
  anim

def test_create_bounds_from_res():
  # Create 100p image from the fake brazil bounds
  new_bounds = raster.create_bounds_from_res(100, 200, fake_brazil_bounds())

  # Boundaries should remain the same.
  assert(new_bounds.minx == fake_brazil_bounds().minx)
  assert(new_bounds.maxx == fake_brazil_bounds().maxx)
  assert(new_bounds.miny == fake_brazil_bounds().miny)
  assert(new_bounds.maxy == fake_brazil_bounds().maxy)

  # Pixel size should change.
  assert 0.39 == pytest.approx(new_bounds.pixel_size_x, 0.1)
  assert -0.19 == pytest.approx(new_bounds.pixel_size_y, 0.1)

  # Number of pixels should change.
  assert (new_bounds.raster_size_x == 100)
  assert (new_bounds.raster_size_y == 200)


test_coords_to_indices()
test_is_valid_point()
test_mount_gdrive_local()
smoke_test_gen_isoscape()
test_create_bounds_from_res()

Driver: GTiff/GeoTIFF
Size is 940 x 936 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-73.97513931345594, 5.266527396029211)
Pixel Size = (0.04166666650042771, -0.041666666499513144)


OSError: ignored

#Stamp isoscape

In [6]:
filename = "/content/gdrive/Shared drives/TNC Fellowship 🌳/4. Isotope Research & Signals/code/amazon_rainforest_files/amazon_rasters/variational/ensemble_with_carbon_brisoisorix/fixed_isorix_carbon_ensemble.tiff" #@param

In [7]:
raster.show_stamps(filename)

{'AREA_OR_POINT': 'Area', 'P_VALUE_THRESHOLD': '0.6'}

In [8]:
metadata_name = "TEST"
metadata_value = 0.55
raster.stamp_isoscape(filename, metadata_name, metadata_value)

In [9]:
raster.show_stamps(filename)

{'AREA_OR_POINT': 'Area', 'P_VALUE_THRESHOLD': '0.6', 'TEST': '0.55'}

In [10]:
metadata_name = "TEST"

In [11]:
raster.del_stamp(filename, metadata_name)

In [12]:
raster.show_stamps(filename)

{'AREA_OR_POINT': 'Area', 'P_VALUE_THRESHOLD': '0.6'}